In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .appName("crear_one_big_table") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-11-19T00:29:17,429 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-11-19T00:29:18,741 WARN [Thread-4] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2023-11-19T00:29:18,743 WARN [Thread-4] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2023-11-19T00:29:18,743 WARN [Thread-4] org.apache.spark.util.Utils - Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [6]:
spark.sql("""
    select count(1)
    from 
        datalake.actividades_eventos ae 
            INNER JOIN datalake.actividades a ON (ae.id_actividad = a.id_actividad)
            INNER JOIN datalake.usuarios u ON (a.id_usuario = u.id_usuario) 
       --     INNER JOIN datalake.dispositivos d ON (a.id_dispositivo = d.id_dispositivo)
       --     INNER JOIN datalake.planes_de_entrenamiento pe ON (a.id_plan = pe.id_plan)
""").show()

+--------+
|count(1)|
+--------+
|    3990|
+--------+



In [9]:
spark.sql("select count(*), count(distinct id_usuario) from datalake.usuarios").show()

+--------+--------------------------+
|count(1)|count(DISTINCT id_usuario)|
+--------+--------------------------+
|    1000|                       197|
+--------+--------------------------+



In [10]:
spark.sql("select * from datalake.usuarios order by id_usuario").show()

+--------------------+--------+-----------+--------------------+----------+--------------+-------------+------------+----------+-----------+-----------+
|          id_usuario|  nombre|   apellido|               email|      sexo|          pais|    actividad|subscripcion|fecha_subs|recha_renov|ultimo_pago|
+--------------------+--------+-----------+--------------------+----------+--------------+-------------+------------+----------+-----------+-----------+
|0075f59b-4ec7-49b...|   Becca|    Schwand|bschwandm@deviant...|  Bigender|        Russia|personalizado|       anual|2023-01-27|       null| 2022-12-15|
|0075f59b-4ec7-49b...|   Prudi|    Vanyard|pvanyardbc@woothe...|Polygender|   North Korea|     triatlon|       anual|2023-05-03|       null| 2023-03-01|
|0075f59b-4ec7-49b...|   Traci|   Cansdill|tcansdillm4@quant...|    Female|      Colombia|     triatlon|      basica|2022-12-24|       null| 2023-03-27|
|0075f59b-4ec7-49b...|  Tallie|  Ellingham|tellingham56@kick...|    Female|       